In [170]:
stock_symbols = ['ASIANPAINT.NS', 'BAJAJ-AUTO.NS', 'BAJFINANCE.NS', 'HCLTECH.NS',
                 'INFY.NS', 'TCS.NS', 'TECHM.NS']
weights_gmv = [0.1966477, 0.13984027, 0.21313399, 0.06534991, 0.108151, 0.18365712, 0.09322002]
weights_msr = [4.24689410e-13, 0.00000000e+00, 2.84356192e-01, 6.29304990e-12, 1.47187965e-01, 4.75904053e-01, 9.25517896e-02]

stock_dict = dict(zip(stock_symbols, weights_gmv))

In [171]:
import pandas as pd
real_prices = pd.read_csv("backtesting_data.csv")

In [172]:
real_prices.rename(columns={'_time': 'Date'}, inplace=True)
real_prices['Date'] = pd.to_datetime(real_prices['Date']).dt.date
real_prices.set_index('Date', inplace=True)

In [173]:
real_prices = real_prices.apply(pd.to_numeric)
real_returns = real_prices.pct_change()
real_returns = real_returns.dropna()

In [174]:
if set(real_returns.columns) != set(stock_dict.keys()):
    raise ValueError("Column names in the DataFrame do not match the keys in the weights dictionary.")
weights_series = pd.Series(stock_dict)
real_returns['Combined Return'] = real_returns.dot(weights_series)
combined_returns_df = real_returns.reset_index()[['Date', 'Combined Return']]
combined_returns_df['Date'] = pd.to_datetime(combined_returns_df['Date']).dt.date
combined_returns_df.set_index('Date', inplace=True)

In [175]:
import numpy as np

def run_cppi(risky_r, safe_r=None, m=3, start=1000, floor=0.8, riskfree_rate=0.03, drawdown=None):
    dates = risky_r.index
    n_steps = len(dates)
    account_value = start
    floor_value = start*floor
    peak = account_value
    if isinstance(risky_r, pd.Series): 
        risky_r = pd.DataFrame(risky_r, columns=["R"])

    if safe_r is None:
        safe_r = pd.DataFrame().reindex_like(risky_r)
        safe_r.values[:] = riskfree_rate/245
    account_history = pd.DataFrame().reindex_like(risky_r)
    risky_w_history = pd.DataFrame().reindex_like(risky_r)
    cushion_history = pd.DataFrame().reindex_like(risky_r)
    floorval_history = pd.DataFrame().reindex_like(risky_r)
    peak_history = pd.DataFrame().reindex_like(risky_r)

    for step in range(n_steps):
        if drawdown is not None:
            peak = np.maximum(peak, account_value)
            floor_value = peak*(1-drawdown)
        cushion = (account_value - floor_value)/account_value
        risky_w = m*cushion
        risky_w = np.minimum(risky_w, 1)
        risky_w = np.maximum(risky_w, 0)
        safe_w = 1-risky_w
        risky_alloc = account_value*risky_w
        safe_alloc = account_value*safe_w
        account_value = risky_alloc*(1+risky_r.iloc[step]) + safe_alloc*(1+safe_r.iloc[step])
        cushion_history.iloc[step] = cushion
        risky_w_history.iloc[step] = risky_w
        account_history.iloc[step] = account_value
        floorval_history.iloc[step] = floor_value
        peak_history.iloc[step] = peak
    risky_wealth = start*(1+risky_r).cumprod()
    backtest_result = {
        "Wealth": account_history,
        "Risky Wealth": risky_wealth, 
        "Risk Budget": cushion_history,
        "Risky Allocation": risky_w_history,
        "m": m,
        "start": start,
        "floor": floor,
        "risky_r":risky_r,
        "safe_r": safe_r,
        "drawdown": drawdown,
        "peak": peak_history,
        "floor": floorval_history
    }
    return backtest_result

In [179]:
run_cppi(combined_returns_df, safe_r= None, m=3, start=1000, floor=0.7, riskfree_rate=0.06971, drawdown=None)

{'Wealth':             Combined Return
 Date                       
 2023-01-03      1005.171077
 2023-01-04       999.539535
 2023-01-05       985.788559
 2023-01-06       972.449813
 2023-01-09       983.807803
 ...                     ...
 2023-12-22      1229.071548
 2023-12-26      1226.905715
 2023-12-27      1241.007253
 2023-12-28      1240.761499
 2023-12-29      1242.448929
 
 [244 rows x 1 columns],
 'Risky Wealth':             Combined Return
 Date                       
 2023-01-03      1005.714027
 2023-01-04       999.499614
 2023-01-05       984.172983
 2023-01-06       968.819446
 2023-01-09       982.229973
 ...                     ...
 2023-12-22      1235.882625
 2023-12-26      1233.704789
 2023-12-27      1247.884473
 2023-12-28      1247.637358
 2023-12-29      1249.334139
 
 [244 rows x 1 columns],
 'Risk Budget':             Combined Return
 Date                       
 2023-01-03         0.300000
 2023-01-04         0.303601
 2023-01-05         0.299678
 2023-